In [8]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureTextCompletion,
    OpenAITextCompletion,
)
from semantic_kernel.orchestration.context_variables import ContextVariables

In [9]:
useAzureOpenAI = True

kernel = sk.Kernel()

# Configure AI service used by the kernel. Load settings from the .env file.
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service(
        "dv", AzureTextCompletion(deployment, endpoint, api_key)
    )
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service(
        "dv", OpenAITextCompletion("text-davinci-003", api_key, org_id)
    )

In [10]:
skills_directory = "skills"

reviewer_skill = kernel.import_semantic_skill_from_directory(
    skills_directory, "Reviewer"
)

review_function = reviewer_skill["Review"]

In [11]:
code_block = ""
with open("tests\\data\\method-UpdateOrderAsync.cs", "+r") as file:
    code_block = file.read()
print(code_block)

public async Task UpdateOrderAsync(int basketId, Address shippingAddress)
{
    var basket = await _basketRepository.GetByIdAsync(basketId);
    Guard.Against.NullBasket(basketId, basket);
    var items = new List<OrderItem>();
    foreach (var item in basket.Items)
    {
        var catalogItem = await _itemRepository.GetByIdAsync(item.CatalogItemId);
        var itemOrdered = new CatalogItemOrdered(catalogItem.Id, catalogItem.Name, catalogItem.PictureUri);
        var orderItem = new OrderItem(itemOrdered, item.UnitPrice, item.Quantity);
        items.AddAsync(orderItem);
    }
    var order = new Order(basket.BuyerId, shippingAddress, items);

    await _orderRepository.UpdateAsync(order);
}


In [12]:
import json

# The "input" variable in the prompt is set by "content" in the ContextVariables object.
context_variables = ContextVariables(content=code_block, variables={"lang": "C#"})
result = review_function(variables=context_variables)

feedback = json.loads(result.result)
print(json.dumps(feedback, indent=2))

JSONDecodeError: Expecting value: line 1 column 2 (char 1)